In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, Binary

from Utils import dataPreparation, drawGraph

In [2]:

data = dataPreparation("Dyadic_COW_4.0.csv")

# Create an empty directed graph
G = nx.Graph()

# Add edges to the graph
for idx, row in data.iterrows():
    G.add_edge(row["ccode1"], row["ccode2"], weight=row["flow1_pct_change"])

# create a dictionary between country codes and country names
ccodes = data['ccode1'].unique()
cnames = data['importer1'].unique()

ccode_dict = dict(zip(ccodes, cnames))
ccode_dictReversed = {v: k for k, v in ccode_dict.items()}

# Draw the graph
print("Plotting the graph")
drawGraph(G, "graph.png", ccode_dict)

# print the number of nodes
print("Nodes present in the graph: ", len(G.nodes))

# create a method to remove a node from the graph and his edges
def remove_node(G, node):
    # remove the node from the graph
    G.remove_node(node)

# # list of removable nodes
# list = ["San Marino", "East Timor", "Monaco", "Yugoslavia", "Andorra", "Cape Verde", "Marshall Islands", "Vietnam",
#         "Sao Tome and Principe", "Kiribati", "Tuvalu", "Nauru", "Palau", "Solomon Islands", "Vanuatu", "Comoros",
#         "Seychelles", "Mauritius", "Maldives", "Liechtenstein", "Federated States of Micronesia", "Vietnam",
#         "Liechtenstein", "South Sudan", "Kazakhstan", "Venezuela", "Nicaragua"]

# list of foundamental nodes
f_list = ["United States of America", "Italy", "Germany", "Canada", "China", "Russia", "Japan", "Brazil", "India",
          "France", "United Kingdom", "Australia", "Spain", "Mexico", "South Korea", "Netherlands", "Turkey",
          "Saudi Arabia", "Indonesia", "Switzerland", "Sweden", "Norway", "Denmark", "Finland", "Belgium",
          "Austria", "Poland", "Czech Republic", "Portugal", "Greece", "Ireland", "Hungary", "Romania", "Bulgaria",
          "Croatia", "Slovenia", "Slovakia", "Estonia", "Latvia", "Lithuania", "Cyprus", "Malta", "Luxembourg",
          "Iceland", "New Zealand", "Singapore", "Taiwan", "South Africa", "Argentina", "Chile", "Colombia", "Peru",
          "Thailand", "Philippines", "Malaysia"]

# keep only the nodes that are in the list and remove the others
nodesToKeep = []
nodesToRemove = []
for node in f_list:
    node = ccode_dictReversed[node]
    nodesToKeep.append(node)

# iterate over the nodes and remove the ones that are not in the list
for node in G.nodes:
    if node not in nodesToKeep:
        nodesToRemove.append(node)

# remove nodes
for node in nodesToRemove:
    try:
        remove_node(G, node)
    except:
        pass

# print the number of nodes
print("Remaining nodes: ", len(G.nodes))
print("Remaining edges: ", len(G.edges))

# recalculate ccode_dict, accounting for the removed nodes
new_ccode_dict = {k: v for k, v in ccode_dict.items() if k in G.nodes}

# Draw the graph
print("Plotting the graph")
drawGraph(G, "prunedGraph.png", new_ccode_dict)

Plotting the graph
Nodes present in the graph:  195
Remaining nodes:  55
Remaining edges:  1485
Plotting the graph


In [10]:
best_solution = {0: ['United States of America', 'Canada', 'Colombia', 'Peru', 'Argentina', 'United Kingdom', 'Poland', 'Hungary', 'Czech Republic', 'Croatia', 'Slovenia', 'Bulgaria', 'Russia', 'Estonia', 'Latvia', 'Lithuania', 'Finland', 'Denmark', 'Turkey', 'China', 'Taiwan', 'Japan', 'India', 'Thailand', 'Malaysia', 'Singapore', 'Philippines', 'Indonesia', 'Australia', 'New Zealand'], 1: ['Mexico', 'Colombia', 'Chile', 'Ireland', 'Netherlands', 'Belgium', 'Luxembourg', 'Portugal', 'Hungary', 'Czech Republic', 'Slovakia', 'Italy', 'Malta', 'Romania', 'Estonia', 'Latvia', 'Lithuania', 'Finland', 'Denmark', 'Iceland', 'Taiwan', 'Malaysia', 'Singapore'], 2: ['Canada', 'Mexico', 'Chile', 'Netherlands', 'Belgium', 'Luxembourg', 'France', 'Switzerland', 'Spain', 'Portugal', 'Germany', 'Poland', 'Austria', 'Slovakia', 'Italy', 'Malta', 'Slovenia', 'Greece', 'Cyprus', 'Bulgaria', 'Russia', 'Sweden', 'Norway', 'Iceland', 'South Africa', 'Saudi Arabia', 'South Korea', 'Thailand', 'Philippines', 'Indonesia']}

best_solution = {0: ["United States of America", "Italy", "Germany", "Canada", "China", "Russia", "Japan", "Brazil", "India",
          "France", "United Kingdom", "Australia", "Spain", "Mexico", "South Korea", "Netherlands", "Turkey"],1: [
          "Saudi Arabia", "Indonesia", "Switzerland", "Sweden", "Norway", "Denmark", "Finland", "Belgium",
          "Austria", "Poland", "Czech Republic", "Portugal", "Greece", "Ireland", "Hungary", "Romania", "Bulgaria"],2:[
          "Croatia", "Slovenia", "Slovakia", "Estonia", "Latvia", "Lithuania", "Cyprus", "Malta", "Luxembourg",
          "Iceland", "New Zealand", "Singapore", "Taiwan", "South Africa", "Argentina", "Chile", "Colombia", "Peru",
          "Thailand", "Philippines", "Malaysia"]}

In [11]:
from Utils import drawSinglePlot

#convert best solution to ccode
for key in best_solution:
    for i in range(len(best_solution[key])):
        best_solution[key][i] = ccode_dictReversed[best_solution[key][i]]

# we have the best solution, now we need to plot the graph with the best solution. Use a different color for each cluster
color_map = []
for node in G.nodes:
    if node in best_solution[0]:
        color_map.append('blue')
    elif node in best_solution[1]:
        color_map.append('red')
    elif node in best_solution[2]:
        color_map.append('green')
    else:
        color_map.append('black')

# Draw the graph
print("Plotting the graph")
drawSinglePlot(G, "clustered_prunedGraph.png", new_ccode_dict, color_map)

Plotting the graph
